In [59]:
import warnings
warnings.filterwarnings("ignore")
#warnings.catch_warnings()
import robin_stocks as r
from datetime import datetime
from dateutil import tz
import pandas as pd
from operator import itemgetter, attrgetter, methodcaller

from_zone = tz.tzutc()
to_zone = tz.tzlocal()
def utcToLocal(strDate):
    utc = datetime.strptime(strDate, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

import configparser
config = configparser.RawConfigParser()
configFilePath = '/Users/philipmassey/.tokens/robinhood.cfg'
config.read(configFilePath)
rhuser = config.get('login', 'user')
rhpwd = config.get('login', 'pwd')
login = r.login(rhuser,rhpwd)

def diffMaxHighLow(df, dfstrike_prices):
    print('Strike\tVolume\tHigh\tLow\tChange\tPercent')
    for idx,row in  dfstrike_prices.iterrows():
        strike_price = row.strike_price
        volume = row.volume
        df_stpr = df[(df.strike_price == strike_price)]
        high_price = max(df_stpr.high_price,default=0)
        low_price = min(df_stpr.high_price, default=0)
        diff = round((high_price - low_price),2)
        percent = 0 if low_price==0 else round(diff/low_price,2)
        print('${:.2f}\t{:,.0f}\t{}\t{}\t{}\t{:,.0%}'.format(strike_price,volume,high_price,low_price,diff,percent))
  
def changesForExpirations(symbol,expiration_dates,optionType,span,volume_limit = 0):
    df_expirationDates = pd.DataFrame()
    for expirationDate in expiration_dates[0:5] :
        print('{}\t'.format(expirationDate),end='')
        options = r.find_options_for_list_of_stocks_by_expiration_date([symbol], expirationDate,optionType) 
        dfoptions = pd.DataFrame((filter(lambda x:x['volume']>volume_limit,options)))
        if dfoptions.empty:  
            print('Volume is 0 for options')
        else:
            dfoptions[['strike_price','high_price','low_price']] = dfoptions[['strike_price','high_price','low_price']].apply(pd.to_numeric)
            dfstrike_prices = dfoptions.sort_values(by='volume',ascending=False)[0:5][['strike_price','volume']]
            dfstrike_prices.sort_values(by='strike_price',inplace=True)
            #dfstrike_prices
            df_all = pd.DataFrame()
            for idx,row in dfstrike_prices.iterrows():
                strike_price = row.strike_price
                #print('Strike price: {}'.format(strike_price))
                df = pd.DataFrame(r.get_option_historicals(symbol,expirationDate,strike_price,optionType,span)['data_points'])
                df['strike_price'] = strike_price
                df_all = pd.concat([df_all,df[['begins_at','strike_price','close_price','high_price','low_price','open_price','volume']]])
            
            df_all[['strike_price','high_price','low_price']] = df_all[['strike_price','high_price','low_price']].apply(pd.to_numeric)
            df_all = df_all[df_all.high_price != 0.01]
            df_all['expiration_date'] = expirationDate
            print()
            diffMaxHighLow(df_all,dfstrike_prices)
            df_expirationDates = pd.concat([df_expirationDates,df_all])
    return df_expirationDates


In [54]:
symbol = 'CXW'
expiration_dates = r.get_chains(symbol)['expiration_dates']
print(expiration_dates)

['2020-06-19', '2020-07-17', '2020-09-18', '2020-11-20', '2020-12-18', '2021-01-15', '2022-01-21']


In [55]:
span = 'day'       
df_expirationDates = changesForExpirations(symbol,expiration_dates,'call',span) 

2020-06-19	*.*.*.*.*.
Strike	Volume	High	Low	Change	Percent
$11.00	4	1.9	1.23	0.67	54%
$12.00	93	1.25	0.53	0.72	136%
$13.00	68	0.4	0.18	0.22	122%
$14.00	26	0.2	0.08	0.12	150%
2020-07-17	*.*.*.*.
Strike	Volume	High	Low	Change	Percent
$12.00	3	1.28	0.88	0.4	45%
$13.00	2	0.78	0.45	0.33	73%
$14.00	10	0.5	0.23	0.27	117%
2020-09-18	*.*.*.*.
Strike	Volume	High	Low	Change	Percent
$13.00	2	1.43	0.9	0.53	59%
$14.00	7	0.9	0.65	0.25	38%
$16.00	2	0.48	0.28	0.2	71%
2020-11-20	*.*.*.
Strike	Volume	High	Low	Change	Percent
$14.00	7	1.18	0.93	0.25	27%
$15.00	9	1.0	0.58	0.42	72%
2020-12-18	*.Volume is 0 for options


In [47]:
span = 'week'       
df_expirationDates = changesForExpirations(symbol,expiration_dates,'call',span) 

2020-06-19	
Strike	Volume	High	Low	Change	Percent
$15.00	20	4.0	2.03	1.97	97%
$17.50	512	1.55	0.33	1.22	370%
$20.00	106	0.15	0.08	0.07	88%
$22.50	2	0.1	0.1	0.0	0%
2020-07-17	
Strike	Volume	High	Low	Change	Percent
$17.50	15	1.98	0.7	1.28	183%
$20.00	47	0.5	0.1	0.4	400%
$22.50	45	0.1	0.08	0.02	25%
2020-08-21	
Strike	Volume	High	Low	Change	Percent
$17.50	2	2.18	0.98	1.2	122%
$20.00	3	0.83	0.3	0.53	177%
2020-11-20	
Strike	Volume	High	Low	Change	Percent
$17.50	1	2.78	1.5	1.28	85%
$20.00	11	1.58	0.7	0.88	126%
$22.50	2	0.75	0.3	0.45	150%
2020-12-18	
Strike	Volume	High	Low	Change	Percent
$15.00	1	4.65	2.98	1.67	56%
$17.50	10	2.85	1.7	1.15	68%
$20.00	32	1.65	0.83	0.82	99%
$25.00	5	0.5	0.23	0.27	117%
$30.00	202	0.2	0.08	0.12	150%


In [48]:
span = 'year'       
df_expirationDates = changesForExpirations(symbol,expiration_dates,'call',span) 

2020-06-19	
Strike	Volume	High	Low	Change	Percent
$15.00	20	6.65	1.33	5.32	400%
$17.50	512	4.05	0.33	3.72	1,127%
$20.00	106	2.55	0.08	2.47	3,088%
$22.50	2	2.55	0.1	2.45	2,450%
2020-07-17	
Strike	Volume	High	Low	Change	Percent
$17.50	15	2.83	0.7	2.13	304%
$20.00	47	2.53	0.1	2.43	2,430%
$22.50	45	0	0	0	0%
2020-08-21	
Strike	Volume	High	Low	Change	Percent
$17.50	2	9.6	0.5	9.1	1,820%
$20.00	3	7.05	0.3	6.75	2,250%
2020-11-20	
Strike	Volume	High	Low	Change	Percent
$17.50	1	5.6	0.68	4.92	724%
$20.00	11	3.53	0.55	2.98	542%
$22.50	2	2.85	0.3	2.55	850%
2020-12-18	
Strike	Volume	High	Low	Change	Percent
$15.00	1	9.0	1.33	7.67	577%
$17.50	10	7.0	0.78	6.22	797%
$20.00	32	5.4	0.3	5.1	1,700%
$25.00	5	3.1	0.13	2.97	2,285%
$30.00	202	2.68	0.08	2.6	3,250%
